In [1]:
import pandas as pd
import pickle

from gensim import corpora, models, matutils
from gensim.models.coherencemodel import CoherenceModel
import scipy.sparse

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from skills.data_preprocess.data.inputs import *
from skills.data_preprocess.data_transform import get_syl_list_2018, get_syl_list_2014, run_data_transform_pipeline

import logging

import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings('ignore')

LDA for 2018 Syllabus Data

In [2]:
syl_18 = get_syl_list_2018()

In [3]:
syl_18_df = pd.DataFrame(syl_18)
syl_18_df.columns = ['description']

In [4]:
data_dtm_18, cv_18 = run_data_transform_pipeline(syl_18_df, False)

In [5]:
data_dtm_18.head()

,,introduction,access,access cell,access content,access control,access introduction,access links,access packages,access wireless,...,yacc,yacc intermediate,yacc top,yarn,yarn mapreduce,z,z buffer,z nature,zone,zone indexes
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


EXPLORATORY DATA ANALYSIS

In [6]:
data_18 = data_dtm_18.transpose()
top_dict_18 = {}
for c in data_18.columns:
    top = data_18[c].sort_values(ascending=False).head(30)
    top_dict_18[c]= list(zip(top.index, top.values))
print(top_dict_18)

{0: [('lattices', 4), ('boolean', 3), ('groups', 3), ('graphs', 3), ('predicate calculus', 2), ('propositional predicate', 2), ('calculus', 2), ('predicate', 2), ('boolean algebra', 2), ('lattices boolean', 2), ('permutations', 2), ('theory', 2), ('subgroups', 2), ('propositional', 2), ('properties', 2), ('trees', 2), ('graph', 2), ('basic', 2), ('algebra', 2), ('ordering permutations', 1), ('hamiltonian graphs', 1), ('hamiltonian', 1), ('function principle', 1), ('generating', 1), ('groups subgroups', 1), ('groups monoids', 1), ('generating function', 1), ('graph theory', 1), ('groups ', 1), ('graphs algorithms', 1)], 1: [('state', 4), ('logic', 3), ('circuits', 3), ('design', 2), ('adder', 2), ('gates', 2), ('sequential circuits', 2), ('sequential', 2), ('registers', 2), ('counters', 2), ('decomposition', 1), ('decoder encoder', 1), ('decoder', 1), ('ring', 1), ('factoring functional', 1), ('product sums', 1), ('encoder', 1), ('products product', 1), ('products', 1), ('encoder code',

In [7]:
for job_role, top_words in top_dict_18.items():
    print(job_role)
    print(', '.join([word for word, count in top_words[0:30]]))
    print('---')

0
lattices, boolean, groups, graphs, predicate calculus, propositional predicate, calculus, predicate, boolean algebra, lattices boolean, permutations, theory, subgroups, propositional, properties, trees, graph, basic, algebra, ordering permutations, hamiltonian graphs, hamiltonian, function principle, generating, groups subgroups, groups monoids, generating function, graph theory, groups , graphs algorithms
---
1
state, logic, circuits, design, adder, gates, sequential circuits, sequential, registers, counters, decomposition, decoder encoder, decoder, ring, factoring functional, product sums, encoder, products product, products, encoder code, nmos, ring johnson, ripple, ripple counters, product, unsigned bcd, unsigned, processor bit, processor, forms k
---
2
memory, representation, cache, point, hardware, cache coherence, protocol, point representation, write, organization, data dependencies, interrupts, micro, control, multiplier, memories, dependencies, programmed, micro programmed,

In [8]:
tdm_18 = data_dtm_18.transpose()
tdm_18.head()

sparse_counts18 = scipy.sparse.csr_matrix(tdm_18)
corpus18 = matutils.Sparse2Corpus(sparse_counts18)

id2word18 = dict((v, k) for k, v in cv_18.vocabulary_.items())
texts18 = [[word for word in list(cv_18.vocabulary_.keys())]]
dictionary18 = corpora.Dictionary(texts18)

In [ ]:
logging.basicConfig(filename='LdaSyllabus18.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO, force=True)

In [10]:
lda7 = models.LdaModel(corpus=corpus18, num_topics=30, id2word=id2word18, passes=60, iterations= 9000, chunksize = 20000, random_state=42)
lda7.print_topics(30,30)

[(0,
  '0.015*"system" + 0.010*"control" + 0.010*"process" + 0.008*"operating system" + 0.008*"operating" + 0.007*"multicore" + 0.007*"organization" + 0.007*"planning" + 0.007*"scheduling" + 0.005*"nature" + 0.005*"types" + 0.005*"" + 0.005*"structure" + 0.005*"memory" + 0.003*"managerial" + 0.003*"span" + 0.003*"nature purpose" + 0.003*"purpose" + 0.003*"leadership" + 0.003*"planning process" + 0.003*"authority" + 0.003*"development" + 0.003*"business" + 0.003*"theory" + 0.003*"basic" + 0.003*"computers" + 0.003*"performance" + 0.003*"core" + 0.003*"supercomputers" + 0.003*"software"'),
 (1,
  '0.000*"peephole optimization" + 0.000*"penalties regularization" + 0.000*"penalties" + 0.000*"perceptron types" + 0.000*"partitioning" + 0.000*"performance increase" + 0.000*"performance metrics" + 0.000*"performance mpi" + 0.000*"performance software" + 0.000*"periodic" + 0.000*"perceptron" + 0.000*"performance" + 0.000*"partitioning markov" + 0.000*"peephole" + 0.000*"pca ica" + 0.000*"pca" +

In [11]:
coherencemodel = CoherenceModel(model=lda7, texts=texts18, dictionary=dictionary18, coherence='c_v')
print(coherencemodel.get_coherence())

0.5630152591433188


In [12]:
f = open('models/lda_final_18.pickle', 'wb')
pickle.dump(lda7, f)
f.close()

In [13]:
f = open('models/lda_final_18.pickle', 'rb')
lda_final_18 = pickle.load(f)
f.close()

TOPIC VISUALIZATION 

In [15]:
# Topic Visualization for syllabus 18 model
pyLDAvis.enable_notebook()
viz18 = pyLDAvis.gensim_models.prepare(lda_final_18, corpus18, dictionary18)
viz18


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
25     0.192993  0.220306       1        1  11.744437
3     -0.231759  0.129540       2        1   9.028884
0     -0.035799  0.068185       3        1   7.076057
22     0.069685 -0.089632       4        1   5.459054
2      0.015685 -0.136260       5        1   5.276507
7      0.018599 -0.051144       6        1   5.245809
6     -0.013742 -0.068404       7        1   5.001418
11     0.014319  0.000607       8        1   4.757115
19     0.002928 -0.011952       9        1   4.696404
4      0.035017 -0.013567      10        1   3.963865
13     0.013153  0.019416      11        1   3.902846
28    -0.109478  0.013966      12        1   3.872341
27     0.023373  0.014939      13        1   3.170653
18     0.004126 -0.015199      14        1   2.621571
23     0.001968 -0.001666      15        1   2.499192
21     0.008536 -0.004462      16        1   2.194094
17    -0.006663  0.018419      17        1   2.133075
8     -0.002803 -0.013041      18        1   2.133075
29     0.001895 -0.008988      19        1   2.041547
10     0.009717 -0.009978      20        1   2.011037
5      0.008298 -0.010611      21        1   1.950020
9      0.000145 -0.006116      22        1   1.950020
26    -0.005519 -0.008365      23        1   1.827983
14    -0.013552 -0.023506      24        1   1.675863
16    -0.002357 -0.001476      25        1   1.461882
12     0.008207 -0.018944      26        1   1.400867
24    -0.007048  0.007632      27        1   0.821274
20     0.000025  0.000100      28        1   0.027704
1      0.000025  0.000100      29        1   0.027704
15     0.000025  0.000100      30        1   0.027704, topic_info=                 Term       Freq      Total Category  logprob  loglift
956              data  20.000000  20.000000  Default  30.0000  30.0000
2640          network  14.000000  14.000000  Default  29.0000  29.0000
3176         programs  11.000000  11.000000  Default  28.0000  28.0000
1350           erlang  10.000000  10.000000  Default  27.0000  27.0000
2666         networks  11.000000  11.000000  Default  26.0000  26.0000
...               ...        ...        ...      ...      ...      ...
2743           number   0.000413   1.723297  Topic30  -8.3896  -0.1458
3078            prime   0.000413   1.043515  Topic30  -8.3896   0.3558
3376        remainder   0.000413   1.043515  Topic30  -8.3896   0.3558
2853  output feedback   0.000413   1.043515  Topic30  -8.3896   0.3558
3455              rsa   0.000413   1.043515  Topic30  -8.3896   0.3558

[1418 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         1  0.155931               
0         2  0.103954               
0         3  0.103954               
0         4  0.051977               
0         5  0.103954               
...     ...       ...            ...
4379      3  0.379876          write
4379      8  0.379876          write
4386      6  0.465877            xml
4386     21  0.465877            xml
4388     21  0.690320  xml retrieval

[1388 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[26, 4, 1, 23, 3, 8, 7, 12, 20, 5, 14, 29, 28, 19, 24, 22, 18, 9, 30, 11, 6, 10, 27, 15, 17, 13, 25, 21, 2, 16])

LDA for 2014 Syllabus

In [16]:
syl_14 = get_syl_list_2014()

In [38]:
syl_14_df = pd.DataFrame(syl_14)
syl_14_df.columns = ['description']

In [39]:
data_dtm_14, cv_14 = run_data_transform_pipeline(syl_14_df, False)

In [41]:
data_14 = data_dtm_14.transpose()
top_dict_14 = {}
for c in data_14.columns:
    top = data_14[c].sort_values(ascending=False).head(30)
    top_dict_14[c]= list(zip(top.index, top.values))
print(top_dict_14)

{0: [('lattices', 4), ('groups', 3), ('theory', 3), ('graphs', 3), ('boolean', 3), ('boolean algebra', 2), ('trees', 2), ('predicate calculus', 2), ('predicate', 2), ('permutations', 2), ('basic', 2), ('graph', 2), ('ordering', 2), ('calculus', 2), ('properties', 2), ('lattices boolean', 2), ('propositional', 2), ('propositional predicate', 2), ('subgroups', 2), ('algebra', 2), ('definitions', 1), ('calculus well', 1), ('eulerian', 1), ('calculus elementary', 1), ('inference', 1), ('degree regular', 1), ('definitions degree', 1), ('degree', 1), ('permutations combinations', 1), ('permutations lexicographical', 1)], 1: [('memory', 5), ('numbers', 4), ('basic', 3), ('operations', 3), ('multiplication', 3), ('introduction', 2), ('introduction basic', 2), ('performance', 2), ('concepts', 2), ('signed', 2), ('fast', 2), ('design', 2), ('modes', 1), ('memory operations', 1), ('hierarchy', 1), ('memory locations', 1), ('hierarchy accessing', 1), ('virtual memory', 1), ('operations characters'

In [42]:
for job_role, top_words in top_dict_14.items():
    print(job_role)
    print(', '.join([word for word, count in top_words[0:30]]))
    print('---')

0
lattices, groups, theory, graphs, boolean, boolean algebra, trees, predicate calculus, predicate, permutations, basic, graph, ordering, calculus, properties, lattices boolean, propositional, propositional predicate, subgroups, algebra, definitions, calculus well, eulerian, calculus elementary, inference, degree regular, definitions degree, degree, permutations combinations, permutations lexicographical
---
1
memory, numbers, basic, operations, multiplication, introduction, introduction basic, performance, concepts, signed, fast, design, modes, memory operations, hierarchy, memory locations, hierarchy accessing, virtual memory, operations characters, modes addition, memory hierarchy, operations addressing, operations introduction, memory common, memory access, , measuring, functional, arithmetic, functional units
---
2
circuits, adder, logic, gates, arithmetic, verilog, sequential, synchronous sequential, logic gates, registers, synchronous, combinational, combinational circuits, fan,

In [43]:
tdm_14 = data_dtm_14.transpose()
tdm_14.head()

sparse_counts14 = scipy.sparse.csr_matrix(tdm_14)
corpus14 = matutils.Sparse2Corpus(sparse_counts14)

id2word14 = dict((v, k) for k, v in cv_14.vocabulary_.items())
texts14 = [[word for word in list(cv_14.vocabulary_.keys())]]
dictionary14 = corpora.Dictionary(texts14)

In [48]:
logging.basicConfig(filename='LdaSyllabus14.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO, force=True)

In [50]:
lda1 = models.LdaModel(corpus=corpus14, num_topics=30, id2word=id2word14, passes=60, iterations= 9000, chunksize = 20000, random_state=42)
lda1.print_topics(30,30)

[(0,
  '0.019*"multimedia" + 0.016*"synchronization" + 0.016*"compression" + 0.006*"video" + 0.006*"principles" + 0.006*"ip" + 0.006*"demand" + 0.003*"data" + 0.003*"applications" + 0.003*"quality" + 0.003*"" + 0.003*"model" + 0.003*"dimensions" + 0.003*"management" + 0.003*"schemes video" + 0.003*"specification methods" + 0.003*"qos ip" + 0.003*"image" + 0.003*"model synchronization" + 0.003*"issues synchronization" + 0.003*"environment multimedia" + 0.003*"protocols prioritized" + 0.003*"application networking" + 0.003*"specification" + 0.003*"data transmission" + 0.003*"environment" + 0.003*"schemes" + 0.003*"applications application" + 0.003*"protocols" + 0.003*"application"'),
 (1,
  '0.011*"project" + 0.011*"students" + 0.009*"marketing" + 0.009*"market" + 0.007*"course" + 0.005*"research" + 0.005*"gained" + 0.005*"opportunity" + 0.005*"projects" + 0.005*"" + 0.005*"new" + 0.005*"work" + 0.005*"expected" + 0.005*"value" + 0.005*"knowledge" + 0.005*"customer" + 0.005*"implementati

In [51]:
coherencemodel = CoherenceModel(model=lda1, texts=texts14, dictionary=dictionary14, coherence='c_v')
print(coherencemodel.get_coherence())

0.5818785665361321


In [52]:
f = open('models/lda_final_14.pickle', 'wb')
pickle.dump(lda1, f)
f.close()

In [53]:
f = open('models/lda_final_14.pickle', 'rb')
lda_final_14 = pickle.load(f)
f.close()

In [54]:
# Topic Visualization for syllabus 14 model
pyLDAvis.enable_notebook()
viz14 = pyLDAvis.gensim_models.prepare(lda_final_14, corpus14, dictionary14)
viz14

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11     0.211921  0.057447       1        1  8.636527
15     0.163738 -0.101034       2        1  8.472430
24    -0.018867 -0.210301       3        1  7.346106
29    -0.133714 -0.039045       4        1  5.890645
3     -0.024344 -0.125184       5        1  5.843709
20    -0.018949 -0.057192       6        1  5.774001
22     0.089888  0.016018       7        1  5.538629
28    -0.058110  0.036687       8        1  5.444764
27     0.036384 -0.016139       9        1  5.257106
13     0.003551 -0.028706      10        1  5.210372
17    -0.003673  0.034325      11        1  4.928479
10    -0.041055  0.089267      12        1  3.825615
1      0.008684  0.069898      13        1  3.003987
14    -0.006298  0.018684      14        1  2.605043
16    -0.031725  0.022825      15        1  2.299792
23     0.010656 -0.006436      16        1  2.135515
9     -0.040784  0.036378      17        1  2.112047
8     -0.049843  0.011923      18        1  2.112047
4      0.000829 -0.004364      19        1  2.065111
5     -0.000618  0.025999      20        1  1.900836
25     0.037511  0.030586      21        1  1.806966
7     -0.008010  0.030019      22        1  1.783498
0     -0.002520  0.011451      23        1  1.572290
18    -0.037935  0.026237      24        1  1.501889
26    -0.007013  0.016265      25        1  1.501889
19    -0.046405  0.018214      26        1  1.337620
21    -0.008325  0.009044      27        1  0.023271
12    -0.008325  0.009044      28        1  0.023271
2     -0.008325  0.009044      29        1  0.023271
6     -0.008325  0.009044      30        1  0.023271, topic_info=                    Term       Freq      Total Category  logprob  loglift
1203                data  45.000000  45.000000  Default  30.0000  30.0000
273         architecture  20.000000  20.000000  Default  29.0000  29.0000
754             circuits   8.000000   8.000000  Default  28.0000  28.0000
100            algorithm  11.000000  11.000000  Default  27.0000  27.0000
5046             testing  11.000000  11.000000  Default  26.0000  26.0000
...                  ...        ...        ...      ...      ...      ...
25                acting   0.000355   1.237871  Topic30  -8.6282   0.2088
26    acting uncertainty   0.000355   1.237871  Topic30  -8.6282   0.2088
27               actions   0.000355   1.239459  Topic30  -8.6282   0.2075
28     actions influence   0.000355   1.239459  Topic30  -8.6282   0.2075
29            activities   0.000355   1.303451  Topic30  -8.6282   0.1572

[1495 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         1  0.094848               
0         2  0.094848               
0         3  0.126465               
0         4  0.031616               
0         5  0.031616               
...     ...       ...            ...
5572      2  0.356095            xml
5572      9  0.356095            xml
5572     10  0.178047            xml
5576      9  0.761310  xml retrieval
5578      6  0.511298           yacc

[1567 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 16, 25, 30, 4, 21, 23, 29, 28, 14, 18, 11, 2, 15, 17, 24, 10, 9, 5, 6, 26, 8, 1, 19, 27, 20, 22, 13, 3, 7])